In [1]:
%load_ext autoreload
%autoreload 2
!pip install plotly

    100% |████████████████████████████████| 41.5MB 762kB/s ta 0:00:011
  Stored in directory: /root/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [2]:
####-----------------Common Stuff-------------------####

In [3]:
# Common
# Import necessary libraries and packages

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
import pandas as pd
import matplotlib.mlab as mlab
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly
from plotly.graph_objs import *
import plotly.graph_objs as go


plotly.tools.set_credentials_file(username='pbd', api_key='nrkkjdZy9uNG8DaoOM2l')


In [4]:
## Common
## Perform pre processing on data i.e. StringIndexing -> OneHotEncoding -> VectorAssembler 

categorical_columns= ['directorName', 'writeName', 'genre', 'runtime', 'year']

### sringIndexer

indexers = [
            StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c))
            for c in categorical_columns
]

### oneHotEncoder

encoders = [OneHotEncoder(dropLast=False,inputCol=indexer.getOutputCol(),
            outputCol="{0}_encoded".format(indexer.getOutputCol())) 
            for indexer in indexers
]

### vectorAssembler

assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders],outputCol="features")

In [5]:
####-----------------Common Stuff-------------------####

In [6]:
####-----------------Start Training-------------------####

In [7]:
##Training
## Load training data with header

data = spark.read.options(header='true', inferschema='true', delimiter=',').csv("2000-2018.csv")  
display(data)
data.cache()
data.show(5)
data.printSchema()




DataFrame[title: string, year: int, avgRating: double, votes: int, directorName: string, writeName: string, runtime: int, genre: string]

+-------------+----+---------+-----+--------------+-----------------+-------+--------------------+
|        title|year|avgRating|votes|  directorName|        writeName|runtime|               genre|
+-------------+----+---------+-----+--------------+-----------------+-------+--------------------+
| Isle of Dogs|2018|      8.2|18140|  Wes Anderson|    Roman Coppola|    101|Adventure,Animati...|
| Isle of Dogs|2018|      8.2|18140|  Wes Anderson|   Kunichi Nomura|    101|Adventure,Animati...|
| Isle of Dogs|2018|      8.2|18140|  Wes Anderson|     Wes Anderson|    101|Adventure,Animati...|
| Isle of Dogs|2018|      8.2|18140|  Wes Anderson|Jason Schwartzman|    101|Adventure,Animati...|
|A Quiet Place|2018|      8.1|68050|John Krasinski|      Bryan Woods|     90| Drama,Horror,Sci-Fi|
+-------------+----+---------+-----+--------------+-----------------+-------+--------------------+
only showing top 5 rows

root
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- 

In [8]:
## Training

## Feed the training data to the pipeline i.e. VectorAssembler -> Pipeline

pipeline = Pipeline(stages=indexers + encoders+[assembler])
model=pipeline.fit(data)
transformed = model.transform(data)

transformed.printSchema()

root
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- avgRating: double (nullable = true)
 |-- votes: integer (nullable = true)
 |-- directorName: string (nullable = true)
 |-- writeName: string (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- directorName_indexed: double (nullable = false)
 |-- writeName_indexed: double (nullable = false)
 |-- genre_indexed: double (nullable = false)
 |-- runtime_indexed: double (nullable = false)
 |-- year_indexed: double (nullable = false)
 |-- directorName_indexed_encoded: vector (nullable = true)
 |-- writeName_indexed_encoded: vector (nullable = true)
 |-- genre_indexed_encoded: vector (nullable = true)
 |-- runtime_indexed_encoded: vector (nullable = true)
 |-- year_indexed_encoded: vector (nullable = true)
 |-- features: vector (nullable = true)



In [9]:
## Training
## Select required columns from the resulting dataframe (training data)

final = transformed.select("features", "avgRating")
final.printSchema()
final.show()

root
 |-- features: vector (nullable = true)
 |-- avgRating: double (nullable = true)

+--------------------+---------+
|            features|avgRating|
+--------------------+---------+
|(9275,[140,3317,8...|      8.2|
|(9275,[140,6879,8...|      8.2|
|(9275,[140,2696,8...|      8.2|
|(9275,[140,4528,8...|      8.2|
|(9275,[1478,6743,...|      8.1|
|(9275,[1478,8394,...|      8.1|
|(9275,[1478,4899,...|      8.1|
|(9275,[930,7441,8...|      8.1|
|(9275,[930,8139,8...|      8.1|
|(9275,[930,7093,8...|      8.1|
|(9275,[2074,2588,...|      7.9|
|(9275,[16,4695,87...|      7.8|
|(9275,[16,2678,87...|      7.8|
|(9275,[522,2431,8...|      7.7|
|(9275,[522,2445,8...|      7.7|
|(9275,[522,7165,8...|      7.7|
|(9275,[522,3746,8...|      7.7|
|(9275,[954,3271,8...|      7.4|
|(9275,[1021,3271,...|      7.4|
|(9275,[1182,7189,...|      7.2|
+--------------------+---------+
only showing top 20 rows



In [10]:
##Training
# To train the model
## Randomly split the data set for testing and training

lr_train,lr_test = final.randomSplit([0.8, 0.2])

## Create a Linear Regression Model with required parameters

lr = LinearRegression(labelCol="avgRating",
                      fitIntercept=True, 
                      maxIter=100, 
                      regParam=0.02, 
                      elasticNetParam=0.02)

## fit the Model to the training dataset

lrModel = lr.fit(lr_train)


In [11]:
##Training
### Collect information about the trained model
## Training summary 

trainingSummary = lrModel.summary

print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

print("Intercept: %s" % str(lrModel.intercept))
print("numIterations: %d" % trainingSummary.totalIterations)

print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
print("Coefficients: %s" % str(lrModel.coefficients))

trainingSummary.predictions.printSchema()
trainingSummary.predictions.show()
training_residuals = trainingSummary.residuals


RMSE: 0.234951
r2: 0.952428
Intercept: 6.533965765409387
numIterations: 101
objectiveHistory: [0.4999564876860057, 0.35619774490246175, 0.07100946954061027, 0.06135485159511546, 0.054073767482482986, 0.051109063308329955, 0.04906147186516218, 0.04751636110083039, 0.04669272268687893, 0.04613455837148201, 0.04581793548721368, 0.04555061279241774, 0.045365380812342586, 0.04519776648153726, 0.045104341019874344, 0.04501851458214046, 0.044953298408235115, 0.04489041685050775, 0.044850732979152146, 0.0448307385619331, 0.04479085141847858, 0.04477486048996353, 0.044752362997427506, 0.044735875381457145, 0.04472001013784247, 0.04470872478430151, 0.04469819524161075, 0.04468926723770783, 0.04468190459480226, 0.044676014684389635, 0.04467185497247561, 0.04466930342019534, 0.044664156338759343, 0.04466237816151824, 0.04465954408462503, 0.04465823561705452, 0.04465630789822835, 0.044655001552190224, 0.044653955528520126, 0.04465301165967729, 0.04465199982377679, 0.04465153362571502, 0.04465093364

+--------------------+---------+------------------+
|            features|avgRating|        prediction|
+--------------------+---------+------------------+
|(9275,[0,2821,881...|      7.0| 6.938841126128502|
|(9275,[0,2821,881...|      7.1| 6.913531671386424|
|(9275,[0,2821,881...|      6.6| 6.832118966996188|
|(9275,[0,3127,881...|      7.0| 6.938841126128502|
|(9275,[0,3127,881...|      7.1| 6.913531671386424|
|(9275,[0,3127,881...|      6.6| 6.832118966996188|
|(9275,[0,3201,877...|      6.5| 6.378798292080902|
|(9275,[0,3201,881...|      7.0| 6.955425551549663|
|(9275,[0,3201,881...|      7.1| 6.930116096807584|
|(9275,[0,3201,881...|      6.6| 6.848703392417349|
|(9275,[0,3323,881...|      6.6|6.6462602209283155|
|(9275,[0,3426,881...|      7.0| 6.938841126128502|
|(9275,[0,3426,881...|      7.1| 6.913531671386424|
|(9275,[0,3426,881...|      6.6| 6.832118966996188|
|(9275,[0,3437,881...|      7.0| 6.938841126128502|
|(9275,[0,3437,881...|      7.1| 6.913531671386424|
|(9275,[0,34

In [12]:
##Training
## Model Evaluation

prediction_and_labels = lrModel.evaluate(final)

print("RMSE: %f" % prediction_and_labels.rootMeanSquaredError)
print("r2: %f" % prediction_and_labels.r2)

prediction_and_labels.predictions.show(5000)

RMSE: 0.332442
r2: 0.904979
+--------------------+---------+------------------+
|            features|avgRating|        prediction|
+--------------------+---------+------------------+
|(9275,[140,3317,8...|      8.2|7.9174266838079195|
|(9275,[140,6879,8...|      8.2| 8.150327666903307|
|(9275,[140,2696,8...|      8.2|  8.03231903725306|
|(9275,[140,4528,8...|      8.2| 7.815911750238037|
|(9275,[1478,6743,...|      8.1| 8.047394084117267|
|(9275,[1478,8394,...|      8.1| 8.047394084117267|
|(9275,[1478,4899,...|      8.1| 7.071672624160145|
|(9275,[930,7441,8...|      8.1|  8.03197374640829|
|(9275,[930,8139,8...|      8.1|  8.03197374640829|
|(9275,[930,7093,8...|      8.1|  8.03197374640829|
|(9275,[2074,2588,...|      7.9| 6.198823694133871|
|(9275,[16,4695,87...|      7.8| 7.745759037176061|
|(9275,[16,2678,87...|      7.8|7.3948421958207495|
|(9275,[522,2431,8...|      7.7| 7.719802756984009|
|(9275,[522,2445,8...|      7.7|7.7236476807206245|
|(9275,[522,7165,8...|      7.7| 7.6

In [16]:
##Training
## Training and Testing Residuals

training_residuals = trainingSummary.residuals
training_residuals.toDF('residuals')
train_resi = training_residuals.write.csv("train.csv")

prediction_residuals = prediction_and_labels.residuals
prediction_residuals.toDF('residuals')
predict_resi = prediction_residuals.write.csv("predict.csv")

In [17]:
####-----------------Training Done-------------------####

In [18]:
## Bridge
## Save the trained Model then load it

lrModel.save("pythonLinearRegression")
Linear_Regression_model = LinearRegressionModel.load("pythonLinearRegression/")

In [19]:
####-----------------Start Prediction-------------------####

In [20]:
## Prediction

## Load new data to be predicted by our model

new_data = spark.read.options(header='true', inferschema='true', delimiter=',').csv("2000-2018.csv")  
display(new_data)
new_data.cache()
new_data.show(500)
new_data.printSchema()

DataFrame[title: string, year: int, avgRating: double, votes: int, directorName: string, writeName: string, runtime: int, genre: string]

+--------------------+----+---------+------+--------------------+--------------------+-------+--------------------+
|               title|year|avgRating| votes|        directorName|           writeName|runtime|               genre|
+--------------------+----+---------+------+--------------------+--------------------+-------+--------------------+
|        Isle of Dogs|2018|      8.2| 18140|        Wes Anderson|       Roman Coppola|    101|Adventure,Animati...|
|        Isle of Dogs|2018|      8.2| 18140|        Wes Anderson|      Kunichi Nomura|    101|Adventure,Animati...|
|        Isle of Dogs|2018|      8.2| 18140|        Wes Anderson|        Wes Anderson|    101|Adventure,Animati...|
|        Isle of Dogs|2018|      8.2| 18140|        Wes Anderson|   Jason Schwartzman|    101|Adventure,Animati...|
|       A Quiet Place|2018|      8.1| 68050|      John Krasinski|         Bryan Woods|     90| Drama,Horror,Sci-Fi|
|       A Quiet Place|2018|      8.1| 68050|      John Krasinski|       

In [21]:
## Prediction

## Feed the New data to the pipeline i.e. VectorAssembler -> Pipeline
pipeline = Pipeline(stages=indexers + encoders+[assembler])
model_new=pipeline.fit(new_data)
transformed_new = model_new.transform(new_data)

transformed_new.printSchema()


root
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- avgRating: double (nullable = true)
 |-- votes: integer (nullable = true)
 |-- directorName: string (nullable = true)
 |-- writeName: string (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- directorName_indexed: double (nullable = false)
 |-- writeName_indexed: double (nullable = false)
 |-- genre_indexed: double (nullable = false)
 |-- runtime_indexed: double (nullable = false)
 |-- year_indexed: double (nullable = false)
 |-- directorName_indexed_encoded: vector (nullable = true)
 |-- writeName_indexed_encoded: vector (nullable = true)
 |-- genre_indexed_encoded: vector (nullable = true)
 |-- runtime_indexed_encoded: vector (nullable = true)
 |-- year_indexed_encoded: vector (nullable = true)
 |-- features: vector (nullable = true)



In [22]:
## Prediction
## Select required columns from the resulting dataframe (New data)

final_new = transformed_new.select("features", "avgRating")
final_new.printSchema()
final_new.show()

root
 |-- features: vector (nullable = true)
 |-- avgRating: double (nullable = true)

+--------------------+---------+
|            features|avgRating|
+--------------------+---------+
|(9275,[140,3317,8...|      8.2|
|(9275,[140,6879,8...|      8.2|
|(9275,[140,2696,8...|      8.2|
|(9275,[140,4528,8...|      8.2|
|(9275,[1478,6743,...|      8.1|
|(9275,[1478,8394,...|      8.1|
|(9275,[1478,4899,...|      8.1|
|(9275,[930,7441,8...|      8.1|
|(9275,[930,8139,8...|      8.1|
|(9275,[930,7093,8...|      8.1|
|(9275,[2074,2588,...|      7.9|
|(9275,[16,4695,87...|      7.8|
|(9275,[16,2678,87...|      7.8|
|(9275,[522,2431,8...|      7.7|
|(9275,[522,2445,8...|      7.7|
|(9275,[522,7165,8...|      7.7|
|(9275,[522,3746,8...|      7.7|
|(9275,[954,3271,8...|      7.4|
|(9275,[1021,3271,...|      7.4|
|(9275,[1182,7189,...|      7.2|
+--------------------+---------+
only showing top 20 rows



In [23]:
## Prediction
## Use the loaded model to predict new data

new_data_predict = Linear_Regression_model.transform(lr_test)
new_data_predict.printSchema()

new_data_predict.show()

root
 |-- features: vector (nullable = true)
 |-- avgRating: double (nullable = true)
 |-- prediction: double (nullable = false)

+--------------------+---------+-----------------+
|            features|avgRating|       prediction|
+--------------------+---------+-----------------+
|(9275,[0,2821,877...|      6.5|6.362213866659743|
|(9275,[0,3127,877...|      6.5|6.362213866659743|
|(9275,[0,3323,881...|      7.0| 6.75298238006063|
|(9275,[0,3323,881...|      7.1|6.727672925318551|
|(9275,[0,3615,881...|      6.6|6.923942983886637|
|(9275,[0,3747,881...|      7.0|6.938841126128502|
|(9275,[0,3793,881...|      6.6|6.923942983886637|
|(9275,[0,4053,881...|      7.1|6.727672925318551|
|(9275,[0,4867,881...|      7.1|6.931483688421568|
|(9275,[0,5060,881...|      7.0|6.938841126128502|
|(9275,[0,6445,881...|      6.6|6.832118966996188|
|(9275,[0,6549,881...|      7.0|6.938841126128502|
|(9275,[0,6609,881...|      7.0|6.938841126128502|
|(9275,[0,6976,881...|      7.1|6.913531671386424|
|(9

In [24]:
####-----------------Prediction Done-------------------####

In [25]:
## Extract relevant information for visualization

actual_and_predicted = new_data_predict.select("avgRating", "prediction")


In [28]:
## Store the predicted values

actual_and_predicted.write.csv("out1.csv")

In [39]:
## Load the stored csv file for visulization

out = spark.read.csv("out1.csv").withColumnRenamed('_c0','label').withColumnRenamed('_c1','prediction')
# out.orderBy('_c2').show(3)
out.show()
out.printSchema()

+-----+-----------------+
|label|       prediction|
+-----+-----------------+
|  6.5|6.362213866659743|
|  6.5|6.362213866659743|
|  7.0| 6.75298238006063|
|  7.1|6.727672925318551|
|  6.6|6.923942983886637|
|  7.0|6.938841126128502|
|  6.6|6.923942983886637|
|  7.1|6.727672925318551|
|  7.1|6.931483688421568|
|  7.0|6.938841126128502|
|  6.6|6.832118966996188|
|  7.0|6.938841126128502|
|  7.0|6.938841126128502|
|  7.1|6.913531671386424|
|  7.1|6.913531671386424|
|  7.0|6.938841126128502|
|  7.0|7.107746641093245|
|  7.6|7.395067741060584|
|  7.6|7.455221403978933|
|  7.3|7.340338916488137|
+-----+-----------------+
only showing top 20 rows

root
 |-- label: string (nullable = true)
 |-- prediction: string (nullable = true)



In [50]:
df = out.toPandas()
df1 = df.iloc[500:540,:]
df1

,label,prediction
500,6.3,5.444829804218778
501,7.1,6.816683621574704
502,6.9,6.65640321110015
503,6.6,5.572530908667318
504,6.6,6.8045513503722415
505,7.8,7.499586382797403
506,7.2,7.33484470615742
507,5.3,5.550743396964926
508,6.2,5.214612435810675
509,5.3,5.215274773472564


In [51]:
table = ff.create_table(df1)
py.iplot(table, filename='table1')

In [53]:
predited_rating = Bar(y=df1.prediction,
                  name='Predicted Rating',
                  marker=dict(color='#206ec1'))

actual_rating = Bar(y=df1.label,
                name='Actual Rating',
                marker=dict(color='#8db72c'))

data = [predited_rating, actual_rating]

layout = Layout(title="Actual and Predicted Ratings",
                xaxis=dict(title='Movies'),
                yaxis=dict(title='Ratings (1 - 10)'))
fig = Figure(data=data, layout=layout)

py.iplot(fig, filename='styled_bar')

In [47]:
trace_actual = go.Scatter(
    y = df.Label,
    name='Actual Rating',
    mode = 'markers',
    marker=dict(color='#8db72c'))


trace_predicted = go.Scatter(
    y = df.Prediction,
    name='Predicted Rating',
    mode = 'markers+lines',
    marker=dict(color='#206ec1'))

layout = Layout(title="Scatter Plot",
                xaxis=dict(title='Movies'),
                yaxis=dict(title='Ratings (1 - 10)'))

fig = Figure(data=data, layout=layout)

data = [trace_actual, trace_predicted]
py.iplot(data, filename='basic-scatter')

In [49]:
residual_df = pd.read_csv("train.csv")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [68]:
trace_actual = go.Scatter(
    y = residual_df.Residual,
    name='Actual Rating',
    mode = 'line',
    marker=dict(color='#8db72c'))


layout = Layout(title="Scatter Plot",
                xaxis=dict(title='Movies'),
                yaxis=dict(title='Ratings (1 - 10)'))

fig = Figure(data=data, layout=layout)

data = [trace_actual]
py.iplot(data, filename='basic-scatter')

In [69]:
residual_df = pd.read_csv("predict1.csv")

In [70]:
trace_actual = go.Scatter(
    y = residual_df.Residual,
    name='Actual Rating',
    mode = 'line',
    marker=dict(color='#206ec1'))


layout = Layout(title="Scatter Plot",
                xaxis=dict(title='Movies'),
                yaxis=dict(title='Ratings (1 - 10)'))

fig = Figure(data=data, layout=layout)

data = [trace_actual]
py.iplot(data, filename='basic-scatter')